In [ ]:
## Tabulate

In [ ]:
import pandas as pd

In [ ]:
from tabulate import tabulate

In [ ]:
# Example data: replace this with your actual model metrics

In [ ]:
data = [

In [ ]:
    {"Model": "Logistic Regression", "Type": "Machine Learning", "Accuracy": 0.74, "Precision": 0.74, "Recall": 0.74, "F1-Score": 0.71, "ROC AUC Score": 0.85},

In [ ]:
    {"Model": "SVM", "Type": "Machine Learning", "Accuracy": 0.76, "Precision": 0.75, "Recall": 0.76, "F1-Score": 0.75, "ROC AUC Score": 0.87},

In [ ]:
    {"Model": "Random Forest", "Type": "Machine Learning", "Accuracy": 0.76, "Precision": 0.75, "Recall": 0.76, "F1-Score": 0.73, "ROC AUC Score": 0.83},

In [ ]:
    {"Model": "CNN", "Type": "Deep Learning", "Accuracy": 0.76, "Precision": 0.75, "Recall": 0.76, "F1-Score": 0.75, "ROC AUC Score": 0.85},